In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preparation

**1. Creating the dataset**

In [2]:
#Import the required libraries
import os
from keras.preprocessing import image

folders = os.listdir('/content/drive/MyDrive/DL/pokemon_project/Train')
# Uncommment the below statement to view the folders
print(folders)

image_data = []
labels = []
count = 0

for ix in folders:
    path = os.path.join("/content/drive/MyDrive/DL/pokemon_project/Train", ix)
    # Uncommment the below statement to view the path to the images and their labels
    print(path, count)
    for im in os.listdir(path):
        try:
            img = image.load_img(os.path.join(path,im), target_size = (224,224))
            img_array = image.img_to_array(img)
            image_data.append(img_array)
            labels.append(count)
        except:
            pass
    count += 1

['Squirtle', 'Spearow', 'Bulbasaur', 'Dratini', 'Meowth', 'Psyduck', 'Fearow', 'Pikachu', 'Aerodactyl', 'Charmander']
/content/drive/MyDrive/DL/pokemon_project/Train/Squirtle 0
/content/drive/MyDrive/DL/pokemon_project/Train/Spearow 1
/content/drive/MyDrive/DL/pokemon_project/Train/Bulbasaur 2
/content/drive/MyDrive/DL/pokemon_project/Train/Dratini 3
/content/drive/MyDrive/DL/pokemon_project/Train/Meowth 4
/content/drive/MyDrive/DL/pokemon_project/Train/Psyduck 5
/content/drive/MyDrive/DL/pokemon_project/Train/Fearow 6
/content/drive/MyDrive/DL/pokemon_project/Train/Pikachu 7


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


/content/drive/MyDrive/DL/pokemon_project/Train/Aerodactyl 8
/content/drive/MyDrive/DL/pokemon_project/Train/Charmander 9


**2. Shuffle the dataset**

In [3]:
import random

combined_dataset = list(zip(image_data, labels))
random.shuffle(combined_dataset)
image_data[:], labels [:] = zip(*combined_dataset)

**3. Final preprocessing of the dataset**

In [4]:
from keras.utils import to_categorical
import numpy as np

X_train = np.array(image_data)
Y_train = np.array(labels)
Y_train = to_categorical(Y_train)

# Model Architecture

**1. Importing the libraries**

In [5]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
import numpy as np

print("Imported Successfully!")

Imported Successfully!


**2. Load the ResNet50 model**

In [6]:
model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3))
print(model.summary())

94765736/94765736 [==============================] - 5s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                      

**3. Building our classifier**

In [7]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense (256, activation = 'relu') (av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(10, activation = 'softmax')(d1)

**4. Perform the feature extractor strategy**

In [8]:
model_new = Model(inputs = model.input, outputs = fc2)
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

# Model Training

**1. Make Predictions**

In [9]:
from keras.applications.resnet50 import preprocess_input

image_path = '/content/drive/MyDrive/DL/pokemon_project/pikachu.png'
img = image.load_img(image_path,target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)
pred = model_new.predict(x)
print(np.argmax(pred))

1/1 [==============================] - 4s 4s/step
9


**2. Compile the new Model**

In [12]:
adam = Adam(lr = 0.00003)
model_new.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

**3. Fine-tune the new model**

In [13]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])
for ix in range(169):
    model_new.layers[ix].trainable = False
    print(model_new.summary())

Streaming output truncated to the last 5000 lines.
 conv4_block5_add (Add)      (None, 14, 14, 1024)         0         ['conv4_block4_out[0][0]',    
                                                                     'conv4_block5_3_bn[0][0]']   
                                                                                                  
 conv4_block5_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block5_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block6_1_conv (Conv2  (None, 14, 14, 256)          262400    ['conv4_block5_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block6_1_bn (BatchNo  (None, 14, 14, 256)          

**4. Train the model**

In [17]:
hist = model_new.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 12)

Epoch 1/12
104/104 [==============================] - 24s 194ms/step - loss: 0.9596 - accuracy: 0.7358
Epoch 2/12
104/104 [==============================] - 19s 181ms/step - loss: 0.6689 - accuracy: 0.8194
Epoch 3/12
104/104 [==============================] - 19s 182ms/step - loss: 0.4449 - accuracy: 0.8758
Epoch 4/12
104/104 [==============================] - 18s 177ms/step - loss: 0.2721 - accuracy: 0.9285
Epoch 5/12
104/104 [==============================] - 19s 179ms/step - loss: 0.2659 - accuracy: 0.9345
Epoch 6/12
104/104 [==============================] - 19s 178ms/step - loss: 0.2184 - accuracy: 0.9358
Epoch 7/12
104/104 [==============================] - 19s 179ms/step - loss: 0.2149 - accuracy: 0.9345
Epoch 8/12
104/104 [==============================] - 19s 178ms/step - loss: 0.2016 - accuracy: 0.9394
Epoch 9/12
104/104 [==============================] - 18s 178ms/step - loss: 0.2370 - accuracy: 0.9297
Epoch 10/12
104/104 [==============================] - 19s 179ms/step - l